In [19]:
import pandas as pd
import plotnine as pn
from plotnine import aes, ggplot
import pathlib
from datetime import datetime, timedelta
from prophet import Prophet

# Import Datasets
The following datasets are used:
- cpi file as supplied by zindi
- currency as supplied by zindi
- jse data as supplied by zindi
-  Monthly credit detail excel released by the reserve bank (https://www.resbank.co.za/en/home/what-we-do/statistics/releases/selected-statistics)
- fuel prices pulled from open price engin

In [20]:
# get directory path
path = str(pathlib.Path().cwd().parent.parent.resolve())

In [21]:
cpi = pd.read_csv(path + '/data/cpi.csv')
currency = pd.read_csv(path + '/data/currency_data.csv')
credit = pd.read_excel(path + '/data/credit.xlsx', header=1)
jse = pd.read_csv(path + '/data/jse_indices.csv')
fuel = pd.read_csv(path + '/data/fuel_df.csv').drop(['Unnamed: 0'], axis=1)
sabor = pd.read_csv(path + '/data/sabor.csv')

Change dates to datetime

In [22]:
# change month to datetime format
cpi['Date'] = pd.to_datetime(cpi['Month']).dt.strftime('%Y-%m')
currency['Date'] = pd.to_datetime(currency['Date']).dt.strftime('%Y-%m')
jse['Date'] = pd.to_datetime(jse['date']).dt.strftime('%Y-%m')
credit['date'] = pd.to_datetime(credit['Date'], format='%b, %Y')
credit['Date'] = pd.to_datetime(credit['date']).dt.strftime('%Y-%m')
fuel['Date'] = pd.to_datetime(fuel['date']).dt.strftime('%Y-%m')
sabor['Date'] = pd.to_datetime(sabor['date']).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_87955/602859498.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.


In [23]:
# Add June CPI
june_cpi = pd.DataFrame({'Category': ['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services'], 
                         'Value': [109.8, 118.3, 110.9, 104.3, 105.4, 107.7, 110.8, 112.3, 99.6, 105.3, 110.4, 110.0, 109.6]})
june_cpi['Date'] = '2023-06'

# Add July CPI
july_cpi = pd.DataFrame({'Category': ['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services'], 
                         'Value': [110.8, 118.5, 111.5, 104.5, 108.4, 108.2, 110.6, 112.6, 99.5, 105.5, 110.4, 110.0, 109.9]})
july_cpi['Date'] = '2023-07'

In [24]:
cpi = pd.concat([cpi, june_cpi, july_cpi])
cpi['Date'] = pd.to_datetime(cpi['Date']).dt.strftime('%Y-%m')

Only select dates from 2022

In [25]:
# change month to datetime format
cpi_new = cpi[cpi['Date'] > '2018']
currency_new = currency[currency['Date'] > '2018']
jse_new = jse[jse['Date'] > '2018']
credit_new= credit[credit['Date'] > '2018']
fuel_new= fuel[fuel['Date'] > '2018']
sabor_new= sabor[sabor['Date'] > '2018']

In the credit and national excels the commas in values need to be stripped and values turned in to intergers

In [26]:
def remove_commas_and_convert_to_float(value):
    return float(value.replace(',', ''))

In [27]:
credit_new = credit_new.drop(['Share of corporations as a % of total credit',
       'Share of corporations as a % of total loans & advances',
       'Share of households as a % of total credit',
       'Share of households as % of total loans & advances', 'Investments'], axis=1)

In [28]:
credit_columns = ['Instalment sale credit', 'Leasing finance',
       'Mortgage advances', 'Overdrafts', 'General loans and advances',
       'Credit card advances', 'Of which: Total to households',
       'Total loans and advances : Households',
       'Claims on the domestic private sector',
       'Total loans and advances (excl. investments & bills)',
       'Bills discounted', 'Instalment sale credit.1', 'Leasing finance.1',
       'Mortgage advances.1', 'Overdrafts.1', 'General loans and advances.1',
       'Credit card advances.1', 'Of which: Total to corporations',
       'Claims on the domestic private sector.1',
       'Total loans and advances : Corporations',]

In [29]:
credit_new[credit_columns] = credit_new[credit_columns].applymap(remove_commas_and_convert_to_float)


# Create Features

In some of the excel sheets we do not have up to date. We will there use the feature from 2 months ao as the feature for now (ie march is the predictor for July)

In [30]:
print(jse_new['Date'].max())
print(credit_new['Date'].max())
print(currency_new['Date'].max())
print(cpi_new['Date'].max())
print(fuel_new['Date'].max())
print(sabor_new['Date'].max())

2023-06
2023-06
2023-06
2023-07
2023-07
2023-06


In [31]:
jse_new['newDate'] = (pd.to_datetime(jse_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
credit_new['newDate'] = (pd.to_datetime(credit_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
currency_new['newDate'] = (pd.to_datetime(currency_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
fuel_new['newDate'] = (pd.to_datetime(fuel_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
sabor_new['newDate'] = (pd.to_datetime(sabor_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_87955/1032560082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_87955/1032560082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
jse_new = jse_new.drop(['Date', 'date'], axis=1)
credit_new = credit_new.drop(['Date', 'date'], axis=1)
currency_new = currency_new.drop(['Date'], axis=1)
fuel_new = fuel_new.drop(['Date', 'date'], axis=1)
sabor_new = sabor_new.drop(['Date', 'date'], axis=1)

## Get mean per Month for jse and currency

In [33]:
credit_mean = credit_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
jse_mean = jse_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
currency_mean = currency_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
fuel_new = fuel_new.groupby(['newDate']).mean().reset_index()
sabor_new = sabor_new.groupby(['newDate']).mean().reset_index()


jse_std = jse_new.groupby(['newDate']).std().reset_index().add_suffix("std")
currency_std = currency_new.groupby(['newDate']).std().reset_index().add_suffix("std")

credit_min = credit_new.groupby(['newDate']).min().reset_index().add_suffix("min")
jse_min = jse_new.groupby(['newDate']).min().reset_index().add_suffix("min")
currency_min = currency_new.groupby(['newDate']).min().reset_index().add_suffix("min")

credit_max = credit_new.groupby(['newDate']).max().reset_index().add_suffix("max")
jse_max = jse_new.groupby(['newDate']).max().reset_index().add_suffix("max")
currency_max = currency_new.groupby(['newDate']).max().reset_index().add_suffix("max")


In [34]:
credit_new = credit_mean.merge(credit_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
credit_new = credit_new.merge(credit_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
credit_new = credit_new.rename(columns={"newDatemean": "newDate"})

In [35]:
jse_new = jse_mean.merge(jse_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
jse_new = jse_new.merge(jse_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
jse_new = jse_new.merge(jse_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
jse_new = jse_new.rename(columns={"newDatemean": "newDate"})

In [36]:
currency_new = currency_mean.merge(currency_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
currency_new = currency_new.merge(currency_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
currency_new = currency_new.merge(currency_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
currency_new = currency_new.rename(columns={"newDatemean": "newDate"})

# Merge data together for features

In [37]:
cpi_all = cpi.drop(['Month'], axis=1).merge(jse_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(credit_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(currency_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(fuel_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(sabor_new, right_on='newDate', left_on='Date', how='left')

In [38]:
cpi_all.shape

(247, 103)

# Model
We will model one index at a time to see what works the best

In [39]:
from models.models import HoltWintersWrapper, ProphetWrapper
from sklearn.metrics import mean_squared_error

In [40]:
feature_columns = list(cpi_all.drop(['Category', 'Date', 'Value', 'newDate'], axis=1).columns)

In [219]:
def combined_model(cpi_all, month):

    hw_cpi_cat = ['Health', 'Education']
    cpi_cat = ['Headline_CPI',
                'Food and non-alcoholic beverages',
                'Alcoholic beverages and tobacco',
                'Clothing and footwear',
                'Housing and utilities',
                'Household contents and services',
                'Transport',
                'Communication',
                'Recreation and culture',
                'Restaurants and hotels ',
                'Miscellaneous goods and services']

    hw_6_results = []
    prophet_multi = []

    # fit hw
    for cat in hw_cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        hw_6 = HoltWintersWrapper(seasonal_periods=6, seasonal='additive')

        hw_6.fit(y=df[df['Date'] < month].sort_values(by='Date')['Value'].values)

        hw_6_results.append(hw_6.predict(forcast=1)[0])

    df_hw_results = pd.DataFrame({'cat':hw_cpi_cat, 'pred':hw_6_results})

    columns_to_drop = ['Category', 'Percentage Change (From Prior Month)', 'newDate']

    df_features = jse_new.merge(credit_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(currency_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(fuel_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(sabor_new, right_on='newDate', left_on='newDate', how='left')

    for cat in cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        # reorder and drop columns
        df.insert(0, 'ds', df.pop('Date'))
        df.insert(1, 'y', df.pop('Value'))
        df = df[['ds', 'y', 'sabor', 'fuel_price', 'USD/ZARmin', 'Credit card advancesmean', 'Total loans and advances : Householdsmean']]

        # model
        prophet = ProphetWrapper(n_changepoints=2, seasonality_mode="multiplicative")
        prophet.fit(df[df['ds'] < month].sort_values(by='ds').sort_values(by='ds'))

        df_predict = pd.DataFrame({'ds': [month]})
        df_predict['ds'] = pd.to_datetime(df_predict['ds']).dt.strftime('%Y-%m')

        df_predict = df_predict.merge(df_features, left_on='ds', right_on='newDate').drop(['newDate'], axis=1)
        df_predict_columns = df.drop(['y'], axis=1).columns
        prophet_multi.append(prophet.predict(df_predict[df_predict_columns])[0])

    df_prophet_results = pd.DataFrame({'cat':cpi_cat, 'pred':prophet_multi})

    df_results = pd.concat([df_hw_results, df_prophet_results])

    return df_results

        

## Get April predictions

In [220]:
df_results_apr = combined_model(cpi_all=cpi_all, month='2023-04')

In [221]:
cpi_apr = cpi_all[cpi_all['Date'] == '2023-04']

In [222]:
df_results_apr = df_results_apr.merge(cpi_apr[['Category', 'Value']], left_on='cat', right_on='Category')

In [223]:
df_results_apr

,cat,pred,Category,Value
0,Health,110.209853,Health,109.6
1,Education,110.429876,Education,110.4
2,Headline_CPI,109.347597,Headline_CPI,109.4
3,Food and non-alcoholic beverages,117.703469,Food and non-alcoholic beverages,117.4
4,Alcoholic beverages and tobacco,109.559992,Alcoholic beverages and tobacco,110.2
5,Clothing and footwear,103.568732,Clothing and footwear,103.7
6,Housing and utilities,104.599350,Housing and utilities,104.6
7,Household contents and services,108.159654,Household contents and services,107.7
8,Transport,112.202267,Transport,113.1
9,Communication,99.549633,Communication,99.8


In [224]:
rmse_apr = mean_squared_error(df_results_apr['pred'], df_results_apr['Value'], squared=False)
print('Apr RMSE prophet multi: ', rmse_apr)

Apr RMSE prophet multi:  0.545230102679257


In [225]:
df_results_apr = df_results_apr[['cat', 'pred']]
df_results_apr.columns = ['ID', 'Value']

month = "April"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_apr = df_results_apr.replace(pred_map)

df_results_apr.to_csv(path + '/submissions/apr_testing.csv', index=False)

## Get May predictions

In [226]:
df_results_may = combined_model(cpi_all=cpi_all, month='2023-05')

In [227]:
cpi_may = cpi_all[cpi_all['Date'] == '2023-05']

In [228]:
df_results_may = df_results_may.merge(cpi_may[['Category', 'Value']], left_on='cat', right_on='Category')

In [229]:
df_results_may

,cat,pred,Category,Value
0,Health,109.876697,Health,110.3
1,Education,110.429480,Education,110.4
2,Headline_CPI,110.031253,Headline_CPI,109.6
3,Food and non-alcoholic beverages,118.403570,Food and non-alcoholic beverages,117.7
4,Alcoholic beverages and tobacco,110.812285,Alcoholic beverages and tobacco,110.6
5,Clothing and footwear,104.003263,Clothing and footwear,104.1
6,Housing and utilities,105.065901,Housing and utilities,104.6
7,Household contents and services,108.079120,Household contents and services,107.5
8,Transport,113.660602,Transport,113.3
9,Communication,99.824732,Communication,99.7


In [230]:
rmse_may = mean_squared_error(df_results_may['pred'], df_results_may['Value'], squared=False)
print('May RMSE prophet multo: ', rmse_may)

May RMSE prophet multo:  0.4065991493974815


In [231]:
df_results_may = df_results_may[['cat', 'pred']]
df_results_may.columns = ['ID', 'Value']

month = "May"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_may = df_results_may.replace(pred_map)

df_results_may.to_csv(path + '/submissions/may_testing.csv', index=False)

## Get June predictions

In [232]:
df_results_june = combined_model(cpi_all=cpi_all, month='2023-06')

In [233]:
cpi_june = cpi_all[cpi_all['Date'] == '2023-06']

In [234]:
df_results_june = df_results_june.merge(cpi_june[['Category', 'Value']], left_on='cat', right_on='Category')

In [235]:
df_results_june

,cat,pred,Category,Value
0,Health,110.531513,Health,110.8
1,Education,110.429396,Education,110.4
2,Headline_CPI,110.912339,Headline_CPI,109.8
3,Alcoholic beverages and tobacco,109.615373,Alcoholic beverages and tobacco,110.9
4,Clothing and footwear,104.835264,Clothing and footwear,104.3
5,Housing and utilities,105.721261,Housing and utilities,105.4
6,Household contents and services,108.622318,Household contents and services,107.7
7,Transport,111.335199,Transport,112.3
8,Communication,100.267649,Communication,99.6
9,Recreation and culture,105.263956,Recreation and culture,105.3


In [236]:
rmse_June = mean_squared_error(df_results_june['pred'], df_results_june['Value'], squared=False)
print('June RMSE prophet multo: ', rmse_June)

June RMSE prophet multo:  0.7163061751305073


In [237]:
df_results_june = df_results_june[['cat', 'pred']]
df_results_june.columns = ['ID', 'Value']

month = "June"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_june = df_results_june.replace(pred_map)

df_results_june.to_csv(path + '/submissions/june_testing.csv', index=False)

# Get predictions for July

In [238]:
df_results_july = combined_model(cpi_all=cpi_all, month='2023-07')

In [239]:
cpi_july = cpi_all[cpi_all['Date'] == '2023-07']

In [240]:
df_results_july = df_results_july.merge(cpi_july[['Category', 'Value']], left_on='cat', right_on='Category')

In [241]:
df_results_july

,cat,pred,Category,Value
0,Health,110.634871,Health,110.6
1,Education,110.429234,Education,110.4
2,Headline_CPI,110.369112,Headline_CPI,110.8
3,Alcoholic beverages and tobacco,111.389341,Alcoholic beverages and tobacco,111.5
4,Clothing and footwear,104.638093,Clothing and footwear,104.5
5,Housing and utilities,106.008393,Housing and utilities,108.4
6,Household contents and services,107.985726,Household contents and services,108.2
7,Transport,112.864867,Transport,112.6
8,Communication,99.460605,Communication,99.5
9,Recreation and culture,105.782768,Recreation and culture,105.5


In [242]:
rmse_July = mean_squared_error(df_results_july['pred'], df_results_july['Value'], squared=False)
print('July RMSE prophet multo: ', rmse_July)

July RMSE prophet multo:  0.7369488890412335


In [243]:
df_results_july = df_results_july[['cat', 'pred']]
df_results_july.columns = ['ID', 'Value']

month = "July"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_july = df_results_july.replace(pred_map)

df_results_july.to_csv(path + '/submissions/july_testing.csv', index=False)

# Get August submission

In [244]:
df_results_aug = combined_model(cpi_all=cpi_all, month='2023-08')

In [245]:
df_results_aug.columns = ['ID', 'Value']

In [246]:
month = 'August'

In [247]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [248]:
df_results_aug = df_results_aug.replace(pred_map)

In [249]:
df_results_aug.to_csv(path + '/submissions/2023-08-21_multi_aug.csv', index=False)